### Подключение библиотек

In [11]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import f_regression,mutual_info_regression
from sklearn.decomposition import PCA,FastICA,TruncatedSVD,NMF
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt



In [12]:
def linear_extrapolation (x):
    X = np.array(x.dropna().index.astype(int)).reshape(-1, 1)
    Y = np.array(x.dropna().values).reshape(-1, 1)
    if X.shape[0] > 0:
        f = LinearRegression().fit(X, Y)
        for i in x.index:
            v = x.loc[i]
            if v != v:
                v = f.predict([[int(i)]])[0][0]
                if v < 0:
                    v = 0
                x.loc[i] = v
    return x
def make_ensemble (x, y):
    return {
        "ridge1": np.argsort(np.abs(Ridge(alpha=0.1).fit(x, y).coef_))[::-1],
        "ridge3": np.argsort(np.abs(Ridge(alpha=0.3).fit(x, y).coef_))[::-1],
        "ridge8": np.argsort(np.abs(Ridge(alpha=0.8).fit(x, y).coef_))[::-1],
        "lasso1": np.argsort(np.abs(Lasso(alpha=0.1).fit(x, y).coef_))[::-1],
        "lasso3": np.argsort(np.abs(Lasso(alpha=0.3).fit(x, y).coef_))[::-1],
        "lasso8": np.argsort(np.abs(Lasso(alpha=0.8).fit(x, y).coef_))[::-1]
    }

### Загрузка и очистка данных
Линейно интерполируем пропуски, а начальные и конечные пропущенные данные экстраполируем по линейному закону. Удаляем пустые параметры

In [13]:
data = pd.read_csv("E:/ittensive/ML selection of factors/Part 3/rosstat.csv",
                    na_values=["-", " - ","...","…"," -"])

In [14]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

In [15]:
data = data.apply(linear_extrapolation, axis=1, result_type="expand")

In [16]:
data["feature"] = features
data.dropna(inplace=True)
features = np.array(data["feature"])

In [17]:
data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)

In [ ]:
data = pd.DataFrame(MinMaxScaler().fit_transform(data))
data.columns = features

### Отбор признаков
Обучим массив моделей для отбора признаков

In [9]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

# -------------------------------------------------------------------------------------------------------------

# Первый набор

In [19]:
super_features = np.zeros(len(features))
subset_size = 50
subset_power = 100
subset_iter = 50
subset_super = 20
for k in range(subset_super):
    subset_features = np.zeros(len(features))
    for j in range(subset_iter):
        features_ensemble = [0]*len(features)
        for f_random in np.random.randint(len(features), size=[subset_power, subset_size]):
            x_subset = data[features[f_random]].copy()
            if y_column in features[f_random]:
                x_subset = x_subset.drop(labels=[y_column], axis=1)
            for y_column_ in y_columns:
                if y_column_ in features[f_random]:
                    x_subset = x_subset.drop(labels=[y_column_], axis=1)
            ensemble = make_ensemble(x_subset, y)
            for e in ensemble:
                i = 0
                for f in ensemble[e]:
                    features_ensemble[f_random[f]] += subset_size - i
                    i += 1
        subset_features[np.argsort(features_ensemble)[::-1][:10]] += 1
    super_features[np.argsort(subset_features)[::-1][:10]] += 1

In [20]:
print (np.argsort(super_features)[::-1][:5])

[170 452 420 173 389]


In [21]:
print (features[np.argsort(super_features)[::-1][:5]])

['6.3. ОБЩЕДОСТУПНЫЕ БИБЛИОТЕКИ 6.3.2. Численность пользователей (тысяч человек)'
 '4.9. ЧИСЛО ПРОФЕССИОНАЛЬНЫХ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ПОДГОТОВКУ КВАЛИФИЦИРОВАННЫХ РАБОЧИХ, СЛУЖАЩИХ в 2000-2015 гг. ;2) (на конец года)'
 '1.10. ОБЩИЕ КОЭФФИЦИЕНТЫ СМЕРТНОСТИ (число умерших на 1000 человек населения)'
 '6.5. ДЕТСКИЙ ОТДЫХ 6.5.2. Численность детей, отдохнувших в них за лето;2) (тысяч человек)'
 'СТРОИТЕЛЬНАЯ ДЕЯТЕЛЬНОСТЬ 15.1. ЧИСЛО ДЕЙСТВУЮЩИХ СТРОИТЕЛЬНЫХ ОРГАНИЗАЦИЙ  (на конец года)']


# Второй набор
1. Важность признаков (feature importace) из решающих деревьев. 
2. Взаимная информация факторов (mutual information) и целевой переменной
3. Корреляция (correlation) факторов и целевой переменной из модели линейная регрессии с L1 и L2 регулиризацией

In [22]:
data = pd.read_csv("E:/ittensive/ML selection of factors/Part 3/rosstat.csv",
                    na_values=["-", " - ","...","…"," -"])

In [23]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

In [24]:
data = data.apply(linear_extrapolation, axis=1, result_type="expand")

In [25]:
data["feature"] = features
data.dropna(inplace=True)
features = data["feature"]

In [26]:
data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)

In [27]:
data = pd.DataFrame(MinMaxScaler().fit_transform(data))
data.columns = features

In [28]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

### Важность признаков (feature importace) из решающих деревьев. 

In [29]:
rfr = RandomForestRegressor(n_estimators=250, random_state=17).fit(x, y)

In [30]:
etr = ExtraTreesRegressor(n_estimators=250, random_state=17).fit(x, y)

### Взаимная информация факторов (mutual information) и целевой переменной

In [31]:
mi = mutual_info_regression(x, y)
mi /= np.max(mi)

### Для линейной регрессии последовательно проверим каждый признак и сохраним его корреляцию

In [32]:
l1 = []
for column in data.columns:
    if column != y_column and column not in y_columns:
        x = np.array(data[column]).reshape(-1, 1)
        y = data[y_column]
        model = Lasso(alpha=0.01).fit(x, y)
        l1.append(model.score(x, y))
l1 = np.array(l1)

In [33]:
l2 = []
for column in data.columns:
    if column != y_column and column not in y_columns:
        x = np.array(data[column]).reshape(-1, 1)
        y = data[y_column]
        model = Ridge(alpha=0.05).fit(x, y)
        l2.append(model.score(x, y))
l2 = np.array(l2)

### Ансамбль моделей
Просуммируем корреляцию, важность признаков и взаимную информацию

In [34]:
ensemble_2 = rfr.feature_importances_ + etr.feature_importances_ + mi + l2 + l1

In [35]:
features[np.argsort(ensemble_2)[::-1][:5]].values

array(['13.2. ОБЪЕМ ОТГРУЖЕННЫХ ТОВАРОВ СОБСТВЕННОГО ПРОИЗВОДСТВА, ВЫПОЛНЕННЫХ РАБОТ И УСЛУГ СОБСТВЕННЫМИ СИЛАМИ ПО ВИДАМ ЭКОНОМИЧЕСКОЙ ДЕЯТЕЛЬНОСТИ 13.2.4.  «Обрабатывающие производства» в соответствии с ОКВЭД2 (в фактически действовавших ценах; миллионов рублей)',
       '18.6. ИСПОЛЬЗОВАНИЕ ПЕРСОНАЛЬНЫХ КОМПЬЮТЕРОВ И СЕТИ ИНТЕРНЕТ В ДОМАШНИХ ХОЗЯЙСТВАХ  18.6.2. Удельный вес домашних хозяйств, имевших доступ к сети Интернет2) (по данным выборочного обследования населения по вопросам использования ИКТ; в процентах от общего числа домашних хозяйств соответствующего субъекта Российской Федерации)',
       '18.6. ИСПОЛЬЗОВАНИЕ ПЕРСОНАЛЬНЫХ КОМПЬЮТЕРОВ И СЕТИ ИНТЕРНЕТ В ДОМАШНИХ ХОЗЯЙСТВАХ  18.6.3. Удельный вес домашних хозяйств, имевших широкополосный доступ к сети Интернет (по данным выборочного обследования населения по вопросам использования ИКТ; в процентах от общего числа домашних хозяйств соответствующего субъекта Российской Федерации)',
       '12.12. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ

# Третий набор
* метод главных компонент (PCA) - 3 модели,
* сингулярное разложение (SVD) - 3 модели,
* анализ независимых компонент (ICA) - 3 модели,
* матричную факторизацию (NMF) - 3 модели.

In [36]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

### Метод главных компонент (PCA) - 3 модели

In [37]:
pca5 = PCA(n_components=5, random_state=42).fit(x)
pca10 = PCA(n_components=10, random_state=42).fit(x)
pca18 = PCA(n_components=18, random_state=42).fit(x)

### Сингулярное разложение (SVD) - 3 модели

In [38]:
svd5 = TruncatedSVD(n_components=5, random_state=42, n_iter=100).fit(x)
svd10 = TruncatedSVD(n_components=10, random_state=42, n_iter=100).fit(x)
svd18 = TruncatedSVD(n_components=18, random_state=42, n_iter=100).fit(x)

### Анализ независимых компонент (ICA) - 3 модели

In [39]:
ica5 = FastICA(n_components=5, random_state=42).fit(x)
ica10 = FastICA(n_components=10, random_state=42).fit(x)
ica18 = FastICA(n_components=18, random_state=42).fit(x)

E:\anaconda\envs\nerual_networks\lib\site-packages\sklearn\decomposition\_fastica.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning,


### Матричную факторизацию (NMF) - 3 модели

In [40]:
nmf5 = NMF(n_components=5, random_state=42, max_iter=1000).fit(x)
nmf10 = NMF(n_components=10, random_state=42, max_iter=1000).fit(x)
nmf18 = NMF(n_components=18, random_state=42, max_iter=5000).fit(x)

E:\anaconda\envs\nerual_networks\lib\site-packages\sklearn\decomposition\_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,
E:\anaconda\envs\nerual_networks\lib\site-packages\sklearn\decomposition\_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,
E:\anaconda\envs\nerual_networks\lib\site-packages\sklearn\decomposition\_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


### Ансамбль моделей
Отнормируем каждый набор важности факторов построчно через MinMax, затем просуммируем важность по всем моделям

In [41]:
ensemble_3 = np.zeros(len(x.columns))

In [42]:
a = MinMaxScaler().fit_transform(np.array(pca5.components_).T)
print (a.T)

[[0.85785658 0.90516015 0.91595028 ... 0.58885223 0.4328719  0.42671716]
 [0.35707202 0.47843557 0.45876395 ... 0.04614578 0.19769678 0.09131371]
 [0.48682805 0.4758213  0.50294996 ... 0.36127293 0.15709916 0.18011024]
 [0.35840713 0.49004016 0.5079901  ... 0.52040737 0.35170694 0.27873669]
 [0.59229627 0.42792351 0.49410853 ... 0.44426293 0.53137184 0.29081794]]


In [43]:
for i,m in enumerate([pca5, pca10, pca18, svd5, svd10, svd18, ica5, ica10, ica18, nmf5, nmf10, nmf18]):
    a = MinMaxScaler().fit_transform(np.array(m.components_).T)
    for c in a.T:
        if i == 0 or i == 3:
            k = 0.2
        elif i == 1 or i == 4:
            k = 0.4
        elif i == 2 or i == 5:
            k = 0.6
        else:
            k = 1
        ensemble_2 += k*np.abs(c)

In [44]:
features[np.argsort(np.abs(ensemble_3))[::-1][:10]].values

array(['4.26. ПРИЕМ НА ОБУЧЕНИЕ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ  (тысяч человек)',
       'ЧИСЛЕННОСТЬ РАБОТНИКОВ ОРГАНОВ МЕСТНОГО САМОУПРАВЛЕНИЯ 2.17.1. Всего',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.6. ЧИСЛЕННОСТЬ БЕЗРАБОТНЫХ (по данным выборочных обследований рабочей силы; тысяч человек)',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.7. ЧИСЛЕННОСТЬ ЗАРЕГИСТРИРОВАННЫХ БЕЗРАБОТНЫХ',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.8. ЧИСЛЕННОСТЬ НЕ ЗАНЯТЫХ ТРУДОВОЙ ДЕЯТЕЛЬНОСТЬЮ ГРАЖДАН, ОБРАТИВШИХСЯ ЗА СОДЕЙСТВИЕМ В ПОИСКЕ ПОДХОДЯЩЕЙ РАБОТЫ В ОРГАНЫ СЛУЖБЫ ЗАНЯТОСТИ НАСЕЛЕНИЯ (тысяч человек)',
       'УРОВЕНЬ БЕЗРАБОТИЦЫ 2.9. 1. Уровень безработицы',
       'УРОВЕНЬ БЕЗРАБОТИЦЫ 2.9.2. Уровень  зарегистрированной безработицы',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.10. ПОТРЕБНОСТЬ В РАБОТНИКАХ, ЗАЯВЛЕННАЯ РАБОТОДАТЕЛЯМИ В ОРГАНЫ СЛУЖБЫ ЗАНЯТОСТИ НАСЕЛЕНИЯ (на конец года; человек)',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.11. НАГРУЗКА НЕЗАНЯТОГО НАСЕЛЕНИЯ, СОСТОЯЩЕГО НА РЕГИСТРАЦИОННОМ УЧЕТЕ В ОРГАНАХ СЛ

In [45]:
ensemble_2[np.argsort(np.abs(ensemble_2))[::-1][:10]]

array([58.71483797, 57.41681412, 55.57416311, 55.45915974, 55.37821787,
       55.11927042, 55.00386144, 54.46576832, 54.31882339, 54.29021642])

# Определение продолжительности жизни

In [63]:
data = pd.read_csv("E:/ittensive/ML selection of factors/Part 3/rosstat.csv",
                    na_values=["-", " - ","...","…"," -"])

In [64]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

In [65]:
data = data.apply(linear_extrapolation, axis=1, result_type="expand")

In [66]:
data["feature"] = features
data.dropna(inplace=True)
features = np.array(data["feature"])

In [67]:
data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)

In [68]:
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = features

In [69]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data[[features[i] for i in [170,160,451,38,452]]]

Сдвинем факторы на год назад: чтобы предсказывать по значениям текущего года - следующий

In [70]:
x_ = x[:-1]
y_ = np.array(y[1:])

### Случайный лес

In [71]:
ensemble_rf = []
for i in range (100):
    ensemble_rf.append(RandomForestRegressor(n_estimators=50, random_state=i).fit(x_, y_))

### Сверхслучайные деревья

In [72]:
ensemble_et = []
for i in range (100):
    ensemble_et.append(ExtraTreesRegressor(n_estimators=50, random_state=i).fit(x_, y_))

### CatBoost

In [73]:
ensemble_cb = []
for i in range (100):
    ensemble_cb.append(CatBoostRegressor(iterations=20, depth=16, random_state=i, verbose=False).fit(x_, y_))

### Формирование предсказания на 2019 год

In [74]:
prediction = 0
for i in range (100):
    prediction += ensemble_rf[i].predict(x[-1:])
    prediction += ensemble_et[i].predict(x[-1:])
    prediction += ensemble_cb[i].predict(x[-1:])
prediction /= 300

In [75]:
print (prediction)

[0.97864546]


In [78]:
data_predict = data.copy()
data_predict[y_column] = np.ones(len(data_predict)) * prediction
data_predict = scaler.inverse_transform(data_predict)
answer = data_predict[0][np.where(features==y_column)][0]

In [81]:
print('ожидаемоя продолжительность жизни:', round(answer), 'года')

ожидаемоя продолжительность жизни: 74 года
